# EDSR

In [2]:
# tensorflow device 확인
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

I0000 00:00:1733465713.288020  244700 gpu_device.cc:2022] Created device /device:GPU:0 with 11539 MB memory:  -> device: 0, name: NVIDIA TITAN Xp, pci bus id: 0000:04:00.0, compute capability: 6.1
I0000 00:00:1733465713.289076  244700 gpu_device.cc:2022] Created device /device:GPU:1 with 11539 MB memory:  -> device: 1, name: NVIDIA TITAN Xp, pci bus id: 0000:05:00.0, compute capability: 6.1
I0000 00:00:1733465713.290010  244700 gpu_device.cc:2022] Created device /device:GPU:2 with 11539 MB memory:  -> device: 2, name: NVIDIA TITAN Xp, pci bus id: 0000:08:00.0, compute capability: 6.1
I0000 00:00:1733465713.293358  244700 gpu_device.cc:2022] Created device /device:GPU:3 with 11539 MB memory:  -> device: 3, name: NVIDIA TITAN Xp, pci bus id: 0000:09:00.0, compute capability: 6.1
I0000 00:00:1733465713.294241  244700 gpu_device.cc:2022] Created device /device:GPU:4 with 11539 MB memory:  -> device: 4, name: NVIDIA TITAN Xp, pci bus id: 0000:83:00.0, compute capability: 6.1
I0000 00:00:173

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14632022366647392205
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 12100173824
 locality {
   bus_id: 1
   links {
     link {
       device_id: 1
       type: "StreamExecutor"
       strength: 1
     }
     link {
       device_id: 2
       type: "StreamExecutor"
       strength: 1
     }
     link {
       device_id: 3
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 2163808062518254773
 physical_device_desc: "device: 0, name: NVIDIA TITAN Xp, pci bus id: 0000:04:00.0, compute capability: 6.1"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 12100173824
 locality {
   bus_id: 1
   links {
     link {
       type: "StreamExecutor"
       strength: 1
     }
     link {
       device_id: 2
       type: "StreamExecutor"
       strength: 1
     }
     link {
       device_id: 3
       type: "Stream

In [1]:
import os
import matplotlib.pyplot as plt

from data import DIV2K, xView
from model.edsr import edsr
from train import EdsrTrainer

%matplotlib inline

2024-12-06 06:15:03.660790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733465703.681774  244700 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733465703.688272  244700 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 06:15:03.710060: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
import matplotlib
print(matplotlib.__version__)

3.9.3


In [10]:
# Number of residual blocks
depth = 16

# Super-resolution factor
scale = 4

# Downgrade operator
downgrade = 'bicubic'

In [11]:
# Location of model weights (needed for demo)
weights_dir = f'weights/xview/edsr-{depth}-x{scale}'
weights_file = os.path.join(weights_dir, 'weights.h5')

os.makedirs(weights_dir, exist_ok=True)

## Datasets

You don't need to download the DIV2K dataset as the required parts are automatically downloaded by the `DIV2K` class. By default, DIV2K images are stored in folder `.div2k` in the project's root directory.

In [12]:
xview_train = xView(scale=scale, subset='train', downgrade=downgrade)
xview_valid = xView(scale=scale, subset='valid', downgrade=downgrade)

In [ ]:
train_ds = xview_train.dataset(batch_size=16, random_transform=True)
valid_ds = xview_valid.dataset(batch_size=1, random_transform=True, repeat_count=1)

Caching decoded images in .xView/caches/xView_train_LR_bicubic_X4.cache ...


2024-12-06 07:23:18.063836: W tensorflow/core/kernels/data/cache_dataset_ops.cc:332] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Cached decoded images in .xView/caches/xView_train_LR_bicubic_X4.cache.
Caching decoded images in .xView/caches/xView_train_HR.cache ...


2024-12-06 07:23:50.235303: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-12-06 07:23:50.236683: W tensorflow/core/kernels/data/cache_dataset_ops.cc:332] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Cached decoded images in .xView/caches/xView_train_HR.cache.


## Training

### Pre-trained models

If you want to skip training and directly run the demo below, download [weights-edsr-16-x4.tar.gz](https://martin-krasser.de/sisr/weights-edsr-16-x4.tar.gz) and extract the archive in the project's root directory. This will create a `weights/edsr-16-x4` directory containing the weights of the pre-trained model.

In [ ]:
trainer = EdsrTrainer(model=edsr(scale=scale, num_res_blocks=depth), 
                      checkpoint_dir=f'.ckpt/xview/edsr-{depth}-x{scale}')

In [ ]:
# Train EDSR model for 300,000 steps and evaluate model
# every 1000 steps on the first 10 images of the DIV2K
# validation set. Save a checkpoint only if evaluation
# PSNR has improved.
trainer.train(train_ds,
              valid_ds.take(10),
              steps=300000, 
              evaluate_every=10, 
              save_best_only=True)

In [ ]:
# Restore from checkpoint with highest PSNR
trainer.restore()

In [ ]:
# Evaluate model on full validation set
psnrv = trainer.evaluate(valid_ds)
print(f'PSNR = {psnrv.numpy():3f}')

In [ ]:
# Save weights to separate location (needed for demo)
trainer.model.save_weights(weights_file)

### DIV2K weights와 비교

In [4]:
weights_dir2 = f'weights/edsr-{depth}-x{scale}'
weights_file2 = os.path.join(weights_dir, 'weights.h5')

NameError: name 'depth' is not defined

In [ ]:
trainer_DIV2K = EdsrTrainer(model=edsr(scale=scale, num_res_blocks=depth), 
                      checkpoint_dir=f'.ckpt/edsr-{depth}-x{scale}')
trainer_DIV2K.model.load_weights(weights_file2)
psnr = trainer_DIV2K.evaluate(valid_ds)
print(f'PSNR = {psnrv.numpy():3f}')

## Demo

In [3]:
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy()

print(f"Number of devices: {strategy.num_replicas_in_sync}")

with strategy.scope():
    model = edsr(scale=4, num_res_blocks=16)
    model2 = edsr(scale=4, num_res_blocks=16)
    model.load_weights('weights/xview/edsr-16-x4-/weights.h5')
    model2.load_weights('weights/edsr-16-x4/weights.h5')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


I0000 00:00:1733465738.867696  244700 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11539 MB memory:  -> device: 0, name: NVIDIA TITAN Xp, pci bus id: 0000:04:00.0, compute capability: 6.1
I0000 00:00:1733465738.868161  244700 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 11539 MB memory:  -> device: 1, name: NVIDIA TITAN Xp, pci bus id: 0000:05:00.0, compute capability: 6.1
I0000 00:00:1733465738.868581  244700 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 11539 MB memory:  -> device: 2, name: NVIDIA TITAN Xp, pci bus id: 0000:08:00.0, compute capability: 6.1
I0000 00:00:1733465738.868984  244700 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 11539 MB memory:  -> device: 3, name: NVIDIA TITAN Xp, pci bus id: 0000:09:00.0, compute capability: 6.1
I0000 00:00:1733465738.869384  244700 gpu_device.cc:2022] Created device /job:localhost/

Number of devices: 8


In [4]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

Physical devices cannot be modified after being initialized


In [5]:
from model import resolve_single
from utils import load_image, plot_sample
import tensorflow as tf

def resolve_and_plot(lr_image_path):
    lr = load_image(lr_image_path)
    sr = resolve_single(model, lr)
    plot_sample(lr, sr)

def resolve_and_plot2(lr_image_path):
    lr = load_image(lr_image_path)
    sr = resolve_single(model2, lr)
    plot_sample(lr, sr)

In [22]:
if tf.test.is_gpu_available():
    print("The following GPU devices are available: %s" % tf.test.gpu_device_name())
    print(1)
    gpus = tf.config.list_physical_devices('GPU')
    print(1)
    tf.config.set_visible_devices(gpus, 'GPU')
    print(1)
else:
    print("GPU is not available")

The following GPU devices are available: /device:GPU:0
1
1
1


I0000 00:00:1733465658.143986  242848 gpu_device.cc:2022] Created device /device:GPU:0 with 11539 MB memory:  -> device: 0, name: NVIDIA TITAN Xp, pci bus id: 0000:04:00.0, compute capability: 6.1
I0000 00:00:1733465658.144600  242848 gpu_device.cc:2022] Created device /device:GPU:1 with 11539 MB memory:  -> device: 1, name: NVIDIA TITAN Xp, pci bus id: 0000:05:00.0, compute capability: 6.1
I0000 00:00:1733465658.145143  242848 gpu_device.cc:2022] Created device /device:GPU:2 with 11539 MB memory:  -> device: 2, name: NVIDIA TITAN Xp, pci bus id: 0000:08:00.0, compute capability: 6.1
I0000 00:00:1733465658.145681  242848 gpu_device.cc:2022] Created device /device:GPU:3 with 11539 MB memory:  -> device: 3, name: NVIDIA TITAN Xp, pci bus id: 0000:09:00.0, compute capability: 6.1
I0000 00:00:1733465658.146246  242848 gpu_device.cc:2022] Created device /device:GPU:4 with 11539 MB memory:  -> device: 4, name: NVIDIA TITAN Xp, pci bus id: 0000:83:00.0, compute capability: 6.1
I0000 00:00:173

In [8]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        details = tf.config.experimental.get_memory_info(gpu.name[17:])
        print(f"GPU: {gpu.name}, Memory Info: {details}")

GPU: /physical_device:GPU:0, Memory Info: {'current': 5628096256, 'peak': 11229229056}
GPU: /physical_device:GPU:1, Memory Info: {'current': 12453888, 'peak': 13043712}
GPU: /physical_device:GPU:2, Memory Info: {'current': 12453888, 'peak': 13043712}
GPU: /physical_device:GPU:3, Memory Info: {'current': 12453888, 'peak': 13043712}
GPU: /physical_device:GPU:4, Memory Info: {'current': 12453888, 'peak': 13043712}
GPU: /physical_device:GPU:5, Memory Info: {'current': 12453888, 'peak': 13043712}
GPU: /physical_device:GPU:6, Memory Info: {'current': 12453888, 'peak': 13043712}
GPU: /physical_device:GPU:7, Memory Info: {'current': 12453888, 'peak': 13043712}


In [7]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [9]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
## 여기
resolve_and_plot('demo/0001.png')

2024-12-06 06:17:59.727998: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:497] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.55GiB (rounded to 2743541760)requested by op Conv2D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-12-06 06:17:59.728080: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1053] BFCAllocator dump for GPU_0_bfc
2024-12-06 06:17:59.728096: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1060] Bin (256): 	Total Chunks: 83, Chunks in use: 82. 20.8KiB allocated for chunks. 20.5KiB in use in bin. 17.1KiB client-requested in use in bin.
2024-12-06 06:17:59.728105: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1060] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-12-06 06:17:59.728

ResourceExhaustedError: {{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[1,64,3228,3320] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

In [ ]:
resolve_and_plot2('')

In [ ]:
import os
import signal

os.kill(os.getpid(), signal.SIGKILL)

: 